---
title: "Projekt z Baz Danych – Raport Końcowy"
format:
  html:
    toc: true
    number-sections: true
    code-fold: true
    code-summary: "Rozwiń kod"
    theme:
      light: flatly
      dark: darkly
page-layout: full
---

# Autorzy

- Jakub Ciągło  
- Mateusz Ćwiek  
- Marek Dworaczyk  
- Rafał Głodek

# Importy

In [ ]:
import mysql.connector
import random
import math
from datetime import date, datetime, timedelta, time
from dateutil.relativedelta import relativedelta
import numpy as np
import pandas as pd
from faker import Faker

# Skryptowe wypełnienie bazy
## Połączenie z bazą danych i wyczyszczenie jej

In [ ]:
con = mysql.connector.connect(
    host = "giniewicz.it",
    user = "team13",
    password = "te@mlie",
    database = "team13"
)

if(con):
    print("Połączenie udane")
else:
    print("Połączenie nieudane")

cursor = con.cursor()

cursor.execute('SET FOREIGN_KEY_CHECKS = 0')

tables = [
    'employee_assignments',
    'feedback',
    'transactions',
    'trip_participants',
    'incidents',
    'costs',
    'trips',
    'trip_types',
    'destinations',
    'launch_stations',
    'spacecraft',
    'rockets',
    'emergency_contacts',
    'clients',
    'employees'
]

for table in tables:
    cursor.execute(f'TRUNCATE TABLE {table}')

cursor.execute('SET FOREIGN_KEY_CHECKS = 1')

con.commit()
cursor.close()
print('Wyczyszczono wszystkie tabele.')

## Wyznaczenie parametrów

In [ ]:
n_employees = 200
clients_per_emp = 3 
n_clients = n_employees * clients_per_emp
min_salary_pln = 14666.00 
avg_salary_pln = 30045.11

current_date = datetime.now() + relativedelta(years=1000)
start_of_business = current_date - relativedelta(years=5)

print(start_of_business)

## Generowanie pracowników

In [ ]:
fake = Faker("pl_PL")
rng = np.random.default_rng()

POSITIONS = [
    ("Pilot statku", 0.07), ("Inżynier systemów", 0.12),
    ("Technik pokładowy", 0.15), ("Kontroler lotów", 0.1),
    ("Astrogator", 0.05), ("Specjalista ds. bezpieczeństwa", 0.05),
    ("Inspektor jakości", 0.06), ("Specjalista HR", 0.04),
    ("Analityk danych", 0.06), ("Marketing kosmiczny", 0.04),
    ("Finanse / Księgowość", 0.06), ("Lekarz pokładowy", 0.05),
    ("Kucharz orbitalny", 0.03), ("Administrator IT", 0.05),
    ("Mechanik rakietowy", 0.07)
]
titles, weights = zip(*POSITIONS)

def random_salary():
    raw = rng.lognormal(mean=math.log(avg_salary_pln), sigma=0.25)
    return round(max(raw, min_salary_pln), 2)

def hire_and_term():
    # Obliczamy różnicę w dniach między start_of_business a current_date - 6 miesięcy
    end_hire_period = current_date - relativedelta(months=6)
    total_days = (end_hire_period - start_of_business).days
    
    # Losujemy liczbę dni i dodajemy do start_of_business, aby uzyskać datę zatrudnienia
    random_days = random.randint(0, total_days)
    hire = start_of_business + timedelta(days=random_days)
    
    # ~15% pracowników już nie pracuje
    if random.random() < 0.15:
        # Data zakończenia pracy musi być co najmniej 3 miesiące po zatrudnieniu
        min_term_date = hire + relativedelta(months=3)
        term_days = (current_date - min_term_date).days
        
        if term_days > 0:
            random_term_days = random.randint(0, term_days)
            term = min_term_date + timedelta(days=random_term_days)
        else:
            term = None  # Jeśli nie ma możliwej daty zakończenia
    else:
        term = None  # Nadal pracuje
    
    return hire, term

emp_rows, used_emails_emp = [], set()
print(f'Generowanie danych dla {n_employees} pracowników...')
for i in range(n_employees):
    first, last = fake.first_name(), fake.last_name()
    position = random.choices(titles, weights)[0]
    salary = random_salary()
    hire, term = hire_and_term()

    base_email = f'{first}.{last}'.lower().replace(' ', '').replace("'", '')
    email = base_email + '@spaceu.com'
    suffix = 1
    while email in used_emails_emp:
        email = f'{base_email}{suffix}@spaceu.com'
        suffix += 1
    used_emails_emp.add(email)

    phone = fake.phone_number()

    if random.random() < 0.02:
        email = None
        phone = None

    emp_rows.append({
        'first_name': first,
        'last_name': last,
        'position': position,
        'salary': salary,
        'hire_date': hire,
        'termination_date': term,
        'email': email,
        'phone': phone
    })

df_emp = pd.DataFrame(emp_rows)
print(f"Wygenerowano {len(df_emp)} rekordów dla pracowników.")

df_emp

## Generowanie klientów

In [ ]:
cli_rows, used_emails_cli = [], set()
domains = ['gmail.com', 'outlook.com', 'yahoo.com', 'protonmail.com']

print(f"\nGenerowanie danych dla {n_clients} klientów...")
for i in range(n_clients):
    first, last = fake.first_name(), fake.last_name()
    base = f"{first}.{last}".lower().replace(" ", "").replace("'", "")
    domain = random.choice(domains)
    email_candidate = f"{base}@{domain}"
    suffix = 1
    while email_candidate in used_emails_cli or email_candidate in used_emails_emp:
        email_candidate = f"{base}{suffix}@{domain}"
        suffix += 1

    phone = fake.phone_number()

    if random.random() < 0.02:
        email_candidate = None
        phone = None

    birth_date = fake.date_of_birth(minimum_age=18, maximum_age=75)
    shifted_birth_date = birth_date.replace(year=birth_date.year + 1000)

    cli_rows.append({
        "first_name": first,
        "last_name": last,
        "date_of_birth": shifted_birth_date,
        "email": email_candidate,
        "phone": phone
    })

df_cli = pd.DataFrame(cli_rows)
print(f"Wygenerowano {len(df_cli)} rekordów dla klientów.")
df_cli

## Zapis pracowników i klientów do bazy (test)

In [ ]:
if con and con.is_connected():
    cursor = con.cursor()
    print("\nPołączono z bazą danych. Rozpoczynanie wstawiania danych...")

    sql_insert_employee = """
    INSERT INTO employees (first_name, last_name, position, salary, hire_date, termination_date, email, phone)
    VALUES (%(first_name)s, %(last_name)s, %(position)s, %(salary)s, %(hire_date)s, %(termination_date)s, %(email)s, %(phone)s)
    """
    employee_records_to_insert = df_emp.to_dict(orient='records')
    
    try:
        cursor.executemany(sql_insert_employee, employee_records_to_insert)
        con.commit()
        print(f"{cursor.rowcount} rekordów wstawiono/zaktualizowano w tabeli employees.")
    except mysql.connector.Error as err:
        print(f"Błąd podczas wstawiania do employees: {err}")
        con.rollback()

    sql_insert_client = """
    INSERT INTO clients (first_name, last_name, date_of_birth, email, phone)
    VALUES (%(first_name)s, %(last_name)s, %(date_of_birth)s, %(email)s, %(phone)s)
    """
    client_records_to_insert = df_cli.to_dict(orient='records')

    try:
        cursor.executemany(sql_insert_client, client_records_to_insert)
        con.commit()
        print(f"{cursor.rowcount} rekordów wstawiono/zaktualizowano w tabeli clients.")
    except mysql.connector.Error as err:
        print(f"Błąd podczas wstawiania do clients: {err}")
        con.rollback()

    # Pokaż, ile w sumie w bazie (używając mysql.connector)
    try:
        cursor.execute("SELECT COUNT(*) FROM employees")
        total_emp = cursor.fetchone()[0]
        print(f"Łącznie w tabeli employees: {total_emp}")

        cursor.execute("SELECT COUNT(*) FROM clients")
        total_cli = cursor.fetchone()[0]
        print(f"Łącznie w tabeli clients: {total_cli}")
    except mysql.connector.Error as err:
        print(f"Błąd podczas odczytu liczby rekordów: {err}")

    print("Zakończono operacje na bazie danych dla tego bloku.")
else:
    print("Błąd: Połączenie z bazą danych ('con') nie jest aktywne lub nie zostało poprawnie zainicjowane.")

# Wprowadzenie pozostałych tabel
## Wygenerowanie statków kosmicznych i rakiet

In [ ]:
def random_date(start_date=start_of_business):
    end_date = start_date + timedelta(days=2*365)
    delta_days = (end_date - start_date).days
    return start_date + timedelta(days=random.randint(0, delta_days))

MANUFACTURERS = ["NASA", "SpaceX", "Blue Origin", "ULA", "Roscosmos", "Arianespace"]

def generate_spacecraft_entries(n=30):
    prefixes = ['Nova', 'Luna', 'Astra', 'Zenith', 'Hyperion', 'Celestia', 'Pulsar', 'Eclipse', 'Titan', 'Vortex']
    suffixes = ['X', 'One', 'Prime', '7', 'NX', 'Infinity', 'Core', 'Ultra', 'VX', 'Edge']
    statuses = ['ACTIVE', 'RETIRED', 'MAINT']

    # Zestaw, w którym będziemy trzymać już użyte nazwy
    used_names = set()
    spacecraft = []

    # Maksymalna liczba możliwych unikalnych kombinacji to len(prefixes)*len(suffixes)=100
    if n > len(prefixes) * len(suffixes):
        raise ValueError(f"Nie da się wygenerować {n} unikalnych nazw (maksymalnie {len(prefixes)*len(suffixes)}).")

    for _ in range(n):
        # Generujemy nazwę, aż będzie unikalna
        while True:
            name = f"{random.choice(prefixes)}-{random.choice(suffixes)}"
            if name not in used_names:
                used_names.add(name)
                break

        manufacturer = random.choice(MANUFACTURERS)
        capacity = random.randint(40, 60)
        status = random.choices(statuses, weights=[0.8, 0.1, 0.1])[0]
        start_date = random_date()
        end_date = None

        if status == 'RETIRED':
            min_end_date = start_date + timedelta(days=365)
            max_end_date = min_end_date + timedelta(days=3*365)
            end_date = min_end_date + timedelta(days=random.randint(0, (max_end_date - min_end_date).days))

        spacecraft.append((capacity, name, manufacturer, start_date, end_date, status))

    return spacecraft


def generate_rocket_entries(n=30):
    prefixes = ['Falcon', 'Vega', 'Atlas', 'Delta', 'Zephyr', 'Aegis', 'Cyclone', 'Talon', 'Nova', 'Strato']
    suffixes = ['IX', 'V', 'Max', 'Pro', 'Ultra', '1000', 'Eon', 'Zero', 'R', 'Edge']
    statuses = ['ACTIVE', 'RETIRED', 'MAINT']

    used_names = set()
    rockets = []

    # Maksymalna liczba możliwych unikalnych kombinacji to znów 10*10 = 100
    if n > len(prefixes) * len(suffixes):
        raise ValueError(f"Nie da się wygenerować {n} unikalnych nazw rakiet (maksymalnie {len(prefixes)*len(suffixes)}).")

    for _ in range(n):
        while True:
            name = f"{random.choice(prefixes)}-{random.choice(suffixes)}"
            if name not in used_names:
                used_names.add(name)
                break

        manufacturer = random.choice(MANUFACTURERS)
        status = random.choices(statuses, weights=[0.8, 0.1, 0.1])[0]
        rockets.append((name, manufacturer, status))

    return rockets


## Stałe do oferty (nazwy itp.) i wprowadzenie ich do bazy

In [ ]:
launch_stations = [
    ("Kennedy Space Center",      "USA", "Cape Canaveral", "ACTIVE"),
    ("Baikonur Cosmodrome",       "Kazakhstan", "Baikonur", "ACTIVE"),
    ("Guiana Space Centre",       "France", "Kourou", "ACTIVE"),
    ("Wenchang",                  "China", "Wenchang", "ACTIVE"),
    ("Space-U Pad Baltic",        "Poland", "Ustka", "PLANNED")
]

rockets = generate_rocket_entries()

spacecraft = generate_spacecraft_entries()

destinations = [
    ("Merkury", "Najmniejsza i najbliższa Słońcu planeta Układu Słonecznego", 0.38, "HIGH"),
    ("Wenus", "Druga planeta od Słońca, o gęstej atmosferze i ekstremalnym cieple", 0.90, "HIGH"),
    ("Mars", "Czwarta planeta od Słońca, znana jako Czerwona Planeta", 0.38, "MEDIUM"),
    ("Jowisz", "Największa planeta w Układzie Słonecznym, gazowy olbrzym", 0.16, "CRITICAL"),
    ("Saturn", "Gazowy olbrzym znany z wyraźnych pierścieni", 0.92, "MEDIUM")
]

trip_types = [
    ("Misja Orbitalna",              "Orbita i obserwacja powierzchni planety",           14, 6_000_000),
    ("Zejście Atmosferyczne",        "Zejście w atmosferę i analiza składu chemicznego",  21, 7_500_000),
    ("Ekspedycja Księżycowa",        "Eksploracja naturalnych satelitów",                 20, 9_000_000),
    ("Obóz Badawczy w Kosmosie",     "Eksperymenty z pokładu bazy orbitalnej",          45,16_000_000),
    ("Przelot przez Pasy Radiacyjne","Przelot przez pasy radiacyjne i pomiary",           40,10_000_000),
    ("Obserwacja Pierścieni",        "Obserwacja struktur pierścieni planetarnych",     20,12_000_000),
    ("Eksplorator Magnetosfery",     "Pomiary pola magnetycznego i zorze",                30, 7_000_000),
    ("Manewr Grawitacyjny",          "Test manewrów grawitacyjnych przy dużych obiektach",10, 5_500_000)
]


def insert_many(q, rows):
    cursor.executemany(q, rows)
    con.commit()

insert_many(
    "INSERT INTO launch_stations (name, country, city, status) VALUES (%s, %s, %s, %s)",
    [(n, c, ci, s) for n, c, ci, s in launch_stations]
)

insert_many(
    "INSERT INTO rockets (name, manufacturer, status) VALUES (%s, %s, %s)",
    [(n, m, s) for n, m, s in rockets]
)

insert_many(
    "INSERT INTO spacecraft (capacity_passengers, name, manufacturer, service_start_date, service_end_date, status) "
    "VALUES (%s, %s, %s, %s, %s, %s)",
    [(cap, n, m, start, end, s) for cap, n, m, start, end, s in spacecraft]
)

insert_many(
    "INSERT INTO destinations (name, description, avg_gravity, hazard_level) VALUES (%s, %s, %s, %s)",
    [(n, d, g, h) for n, d, g, h in destinations]
)

insert_many(
    "INSERT INTO trip_types (name, description, typical_duration_days, base_price) VALUES (%s, %s, %s, %s)",
    [(n, d, dur, price) for n, d, dur, price in trip_types]
)

print("► Załadowano słownikowe tabele.")

## Generowanie trips (50 lotów - można zmienić)

In [ ]:
cursor.execute('SELECT trip_type_id, typical_duration_days FROM trip_types')
trip_types = cursor.fetchall()
trip_type_durations = dict(trip_types)

cursor.execute('SELECT destination_id FROM destinations')
destinations = [row[0] for row in cursor.fetchall()]

cursor.execute("SELECT launch_station_id FROM launch_stations WHERE status = 'ACTIVE'")
launch_stations = [row[0] for row in cursor.fetchall()]

cursor.execute("SELECT spacecraft_id FROM spacecraft WHERE status = 'ACTIVE'")
spacecrafts = [row[0] for row in cursor.fetchall()]

cursor.execute("SELECT rocket_id FROM rockets WHERE status = 'ACTIVE'")
rockets = [row[0] for row in cursor.fetchall()]

statuses = ['completed', 'in progress', 'planned']

# 2) Inicjalizujemy "kalendarze" dla zasobów
spacecraft_schedule     = { sc: [] for sc in spacecrafts }
rocket_schedule         = { rk: [] for rk in rockets }
records_to_insert = []

# Funkcja sprawdzająca kolizję przedziałów
def is_overlapping(new_start, new_end, existing_intervals):
    new_end = new_end or datetime.max
    for ex_start, ex_end in existing_intervals:
        ex_end = ex_end or datetime.max
        if not (new_end < ex_start or ex_end < new_start):
            return True
    return False

# 3) Generujemy 50 wypraw

for _ in range(50):
    trip_type_id = random.choice(list(trip_type_durations.keys()))
    duration = trip_type_durations[trip_type_id]

    destination_id = random.choice(destinations)
    launch_station_id = random.choice(launch_stations)

    status = random.choices(statuses, weights=[0.7, 0.1, 0.2])[0]

    if status == 'completed':
        # Używamy start_of_business jako punktu odniesienia
        base_date = start_of_business + timedelta(days=random.randint(0, 5*365 - duration))
        departure = datetime(
            base_date.year, base_date.month, base_date.day,
            random.randint(0,23), random.randint(0,59), random.randint(0,59)
        )
        return_date = base_date + timedelta(days=duration)
        return_dt = datetime(
            return_date.year, return_date.month, return_date.day,
            random.randint(0,23), random.randint(0,59), random.randint(0,59)
        )

    elif status == 'in progress':
        # Używamy current_date zamiast datetime.now()
        days_back = random.randint(1, duration)
        rand_dt = current_date - timedelta(days=days_back)
        departure = datetime(
            rand_dt.year, rand_dt.month, rand_dt.day,
            random.randint(0,23), random.randint(0,59), random.randint(0,59)
        )
        return_dt = None

    else:  # status == 'planned'
        days_forward = random.randint(1, 365)
        future_dt = current_date + timedelta(days=days_forward)
        departure = datetime(
            future_dt.year, future_dt.month, future_dt.day,
            random.randint(0,23), random.randint(0,59), random.randint(0,59)
        )
        return_dt = None

    new_start = departure
    new_end = return_dt if return_dt is not None else datetime.max.replace(year=current_date.year + 100)

    chosen_spacecraft = None
    chosen_rocket = None

    # 4) Wybór zasobów bez kolizji
    for attempt in range(100):
        sc = random.choice(spacecrafts)
        if is_overlapping(new_start, new_end, spacecraft_schedule[sc]):
            continue

        rk = random.choice(rockets)
        if is_overlapping(new_start, new_end, rocket_schedule[rk]):
            continue

        chosen_spacecraft = sc
        chosen_rocket = rk
        break

    if chosen_spacecraft is None:
        raise RuntimeError(
            "Nie znaleziono wolnego statku + rakiety. "
            "Zmniejsz liczbę lotów lub zasobów."
        )

    # 5) Rejestrujemy zajęcie zasobów
    spacecraft_schedule[chosen_spacecraft].append((new_start, new_end))
    rocket_schedule[chosen_rocket].append((new_start, new_end))

    # 6) Dodajemy wiersz do listy INSERT
    records_to_insert.append((
        trip_type_id,
        destination_id,
        launch_station_id,
        chosen_spacecraft,
        chosen_rocket,
        departure,
        return_dt,
        status
    ))

# 7) Wstawiamy wygenerowane trasy do tabeli trips
cursor.executemany(
    '''
    INSERT INTO trips (
        trip_type_id, destination_id, launch_station_id,
        spacecraft_id, rocket_id, departure_datetime,
        return_datetime, status
    ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
    ''',
    records_to_insert
)
con.commit()

## Generowanie pracowników do lotu (jest 35 na lot- mozna zmienić)

In [ ]:
excluded_positions = [
    'Finanse / Księgowość',
    'Analityk danych',
    'Marketing kosmiczny',
    'Administrator IT',
    'Specjalista HR'
]

# Pobierz pracowników z dozwolonych stanowisk
cursor.execute(
    '''
    SELECT employee_id, position
    FROM employees
    WHERE position NOT IN (%s)
    ''' % ','.join(['%s'] * len(excluded_positions)),
    excluded_positions
)
crew_data = cursor.fetchall()  # lista (employee_id, position)

from collections import defaultdict
position_map = defaultdict(list)
for emp_id, pos in crew_data:
    position_map[pos].append(emp_id)

allowed_positions = list(position_map.keys())

# Pobierz trip_id oraz daty
cursor.execute('SELECT trip_id, departure_datetime, return_datetime FROM trips')
trip_info = cursor.fetchall()

# Harmonogram pracowników
employee_schedule = defaultdict(list)
assignments = []

for trip_id, departure, return_dt in trip_info:
    selected_employees = set()
    trip_assignment = []
    new_start = departure
    new_end = return_dt or datetime.max

    # Po jednej osobie z każdego stanowiska
    for position, emp_list in position_map.items():
        random.shuffle(emp_list)
        for emp_id in emp_list:
            if not is_overlapping(new_start, new_end, employee_schedule[emp_id]):
                selected_employees.add(emp_id)
                employee_schedule[emp_id].append((new_start, new_end))
                trip_assignment.append((trip_id, emp_id))
                break

    # Losowe osoby do uzupełnienia do 35
    remaining_pool = [
        (emp_id, pos)
        for pos, emp_ids in position_map.items()
        for emp_id in emp_ids
        if emp_id not in selected_employees and not is_overlapping(new_start, new_end, employee_schedule[emp_id])
    ]
    additional_needed = 35 - len(trip_assignment)
    if additional_needed > 0:
        random.shuffle(remaining_pool)
        additional_crew = remaining_pool[:additional_needed]
        for emp_id, pos in additional_crew:
            selected_employees.add(emp_id)
            employee_schedule[emp_id].append((new_start, new_end))
            trip_assignment.append((trip_id, emp_id))

    assignments.extend(trip_assignment)

cursor.executemany(
    '''
    INSERT INTO employee_assignments (trip_id, employee_id)
    VALUES (%s, %s)
    ''',
    assignments
)
con.commit()


## Generowanie pasażerów dla lotów


In [ ]:
# Pobierz klientów
cursor.execute('SELECT client_id FROM clients')
client_ids = [row[0] for row in cursor.fetchall()]

# Pobierz loty wraz z przypisanymi statkami, statusem i datami
cursor.execute('SELECT trip_id, spacecraft_id, status, departure_datetime, return_datetime FROM trips')
trips_data = cursor.fetchall()  # lista (trip_id, spacecraft_id, status, departure_datetime, return_datetime)

# Pobierz pojemności statków
cursor.execute('SELECT spacecraft_id, capacity_passengers FROM spacecraft')
capacity_map = dict(cursor.fetchall())  # dict: spacecraft_id -> capacity

# Stwórz strukturę: trip_id -> (capacity, status, departure, return)
trip_info = {
    trip_id: (capacity_map[spacecraft_id], status, departure, return_dt)
    for trip_id, spacecraft_id, status, departure, return_dt in trips_data
}

# Przygotuj dane do INSERT
trip_participants = defaultdict(list)  # trip_id -> list of (client_id, seat_number)
client_trip_assignments = defaultdict(list)  # client_id -> list of (departure, return)

# 1. Zapełnij w pełni loty 'completed' i 'in progress'
filled_trips = [tid for tid, (_, status, _, _) in trip_info.items() if status in ('completed', 'in progress')]
available_clients = client_ids.copy()
random.shuffle(available_clients)

for trip_id in filled_trips:
    capacity, _, departure, return_dt = trip_info[trip_id]
    assigned = 0
    for client_id in available_clients:
        if assigned >= capacity:
            break
        if not is_overlapping(departure, return_dt, client_trip_assignments[client_id]):
            seat_number = len(trip_participants[trip_id]) + 1
            trip_participants[trip_id].append((client_id, seat_number))
            client_trip_assignments[client_id].append((departure, return_dt))
            assigned += 1

# 2. Upewnij się, że każdy klient jest przypisany do co najmniej jednego lotu
all_trip_ids = list(trip_info.keys())
random.shuffle(all_trip_ids)

for client_id in client_ids:
    if not client_trip_assignments[client_id]:
        for trip_id in all_trip_ids:
            capacity, _, departure, return_dt = trip_info[trip_id]
            if len(trip_participants[trip_id]) >= capacity:
                continue
            if not is_overlapping(departure, return_dt, client_trip_assignments[client_id]):
                seat_number = len(trip_participants[trip_id]) + 1
                trip_participants[trip_id].append((client_id, seat_number))
                client_trip_assignments[client_id].append((departure, return_dt))
                break

# 3. Uzupełnij loty 'planned' (częściowo)
for trip_id, (capacity, status, departure, return_dt) in trip_info.items():
    if status != 'planned':
        continue
    remaining_seats = capacity - len(trip_participants[trip_id])
    if remaining_seats <= 0:
        continue
    random.shuffle(client_ids)
    for client_id in client_ids:
        if remaining_seats == 0:
            break
        if not is_overlapping(departure, return_dt, client_trip_assignments[client_id]):
            seat_number = len(trip_participants[trip_id]) + 1
            trip_participants[trip_id].append((client_id, seat_number))
            client_trip_assignments[client_id].append((departure, return_dt))
            remaining_seats -= 1

# Przygotuj dane do INSERT
records_to_insert = [
    (trip_id, client_id, seat_number)
    for trip_id, participants in trip_participants.items()
    for client_id, seat_number in participants
]

cursor.executemany(
    '''
    INSERT INTO trip_participants (trip_id, client_id, seat_number)
    VALUES (%s, %s, %s)
    ''',
    records_to_insert
)
con.commit()


## Emergency contacts

In [ ]:
cursor.execute('SELECT client_id, date_of_birth FROM clients')
clients = cursor.fetchall()

# relacje rozdzielone według płci
# relacje rozdzielone według płci
male_relationships = ['ojciec', 'brat', 'mąż', 'partner', 'przyjaciel', 'syn']
female_relationships = ['matka', 'siostra', 'żona', 'partnerka', 'przyjaciółka', 'córka']
domains = ['gmail.com', 'outlook.com', 'yahoo.com', 'protonmail.com']

records_to_insert = []
used_emails_contacts = set()

for client_id, dob in clients:
    birth_year = dob.year
    if birth_year < 2965:
        available_relationships = [r for r in male_relationships + female_relationships if r not in ['matka', 'ojciec']]
    elif birth_year > 2990:
        available_relationships = [r for r in male_relationships + female_relationships if r not in ['syn', 'córka']]
    else:
        available_relationships = male_relationships + female_relationships

    used_relationships = set()
    for _ in range(2):
        while True:
            gender = random.choice(['male', 'female'])
            first_name = fake.first_name_male() if gender == 'male' else fake.first_name_female()
            last_name = fake.last_name()

            base_email = f"{first_name}.{last_name}".lower().replace(" ", "").replace("'", "")
            domain = random.choice(domains)
            email = f"{base_email}@{domain}"
            suffix = 1
            while email in used_emails_contacts:
                email = f"{base_email}{suffix}@{domain}"
                suffix += 1
            used_emails_contacts.add(email)

            phone = fake.phone_number()

            if random.random() < 0.02:
                email = None
                phone = None

            rel_pool = male_relationships if gender == 'male' else female_relationships
            rel_choices = [r for r in rel_pool if r in available_relationships and r not in used_relationships]

            if rel_choices:
                relationship = random.choice(rel_choices)
                used_relationships.add(relationship)
                break

        records_to_insert.append((
            client_id,
            first_name,
            last_name,
            relationship,
            email,
            phone
        ))

cursor.executemany(
    '''
    INSERT INTO emergency_contacts (
        client_id, first_name, last_name,
        relationship, email, phone
    ) VALUES (%s, %s, %s, %s, %s, %s)
    ''',
    records_to_insert
)


con.commit()


## Generowanie transakcji (transactions) dla uczestników lotów

In [ ]:
# 1. Pobierz dane o uczestnikach lotów wraz z informacjami o ich wyprawie i cenie bazowej
cursor.execute("""
    SELECT 
        tp.trip_id, 
        tp.client_id, 
        t.departure_datetime, 
        tt.base_price
    FROM trip_participants tp
    JOIN trips t ON tp.trip_id = t.trip_id
    JOIN trip_types tt ON t.trip_type_id = tt.trip_type_id
""")
participants_data = cursor.fetchall()  # lista wierszy: (trip_id, client_id, departure_datetime, base_price)

# 2. Definicja metod płatności i statusów transakcji
payment_methods = ['credit_card', 'wire_transfer', 'paypal', 'crypto']
# statusy: tylko przyszłe loty mogą być "pending", przeszłe będą zawsze "completed"
future_statuses = ['completed', 'pending']

records_to_insert = []
now = datetime.now() + relativedelta(years=1000)
start_dt = datetime.combine(start_of_business, time(0, 0, 0))

for trip_id, client_id, departure_dt, base_price in participants_data:
    # 3. Oblicz maksymalną liczbę dni, o które można cofnąć transakcję
    if departure_dt > now:
        days_diff = (departure_dt - start_dt).days
        max_advance_days = min(180, days_diff) if days_diff > 0 else 1
        days_before_departure = random.randint(1, max_advance_days)
    else:
        days_diff = (now - start_dt).days
        max_advance_days = min(180, days_diff) if days_diff > 0 else 1
        days_before_departure = random.randint(1, max_advance_days)

    # 4. Generujemy wstępną datę transakcji
    transaction_date = departure_dt - timedelta(
        days=days_before_departure,
        hours=random.randint(0, 23),
        minutes=random.randint(0, 59),
        seconds=random.randint(0, 59)
    )

    # 5. Gdyby przypadkiem wyszła przed startem działalności → losujemy między start_dt a teraz
    if transaction_date < start_dt:
        earliest = start_dt
        latest = now
        diff_seconds = int((latest - earliest).total_seconds())
        random_offset = random.randint(0, diff_seconds)
        transaction_date = earliest + timedelta(seconds=random_offset)

    # 6. Kwota transakcji (bazowa ±5%)
    variation_pct = random.uniform(-0.05, 0.05)
    amount = round(float(base_price) * (1 + variation_pct), 2)

    # 7. Wybór metody płatności oraz statusu
    payment_method = random.choice(payment_methods)
    if departure_dt > now:
        # lot jeszcze w przyszłości → możliwe 'completed' lub 'pending'
        status = random.choices(future_statuses, weights=[0.7, 0.3])[0]
    else:
        # lot w przeszłości → zawsze 'completed'
        status = 'completed'

    records_to_insert.append((trip_id, client_id, transaction_date, amount, payment_method, status))

# 8. Wstawianie do tabeli transactions
cursor.executemany(
    """
    INSERT INTO transactions (
        trip_id, client_id, transaction_date, amount, payment_method, status
    ) VALUES (%s, %s, %s, %s, %s, %s)
    """,
    records_to_insert
)
con.commit()

## Generowanie opinii dla zakończonych lotów

In [ ]:
# --- 1. Pobierz zakończonych uczestników ---
cursor.execute("""
    SELECT 
        tp.trip_id, 
        tp.client_id, 
        t.return_datetime
    FROM trip_participants tp
    JOIN trips t ON tp.trip_id = t.trip_id
    WHERE t.status = 'completed'
      AND t.return_datetime IS NOT NULL
""")
completed_participants = cursor.fetchall()  # lista: (trip_id, client_id, return_datetime)

# --- 2. Pobierz trip_id, w których były incydenty ---
cursor.execute("SELECT DISTINCT trip_id FROM incidents")
trips_with_incidents = {row[0] for row in cursor.fetchall()}

# --- 3. Listy komentarzy ---
negative_comments = [
    "Lot był bardzo niewygodny, a obsługa niezbyt pomocna.",
    "Paliwo się skończyło, a procedury awaryjne były chaotyczne.",
    "Posiłki były zimne i bez smaku. Ogólnie rozczarowanie.",
    "Sprzęt naukowy na pokładzie nie działał, stracony czas.",
    "Osoby odpowiedzialne za bezpieczeństwo były niekompetentne.",
    "Kabina zbyt mała, bardzo ciasno i duszno.",
    "Trudności z łącznością, nie mogłem porozmawiać z rodziną."
]
neutral_comments = [
    "Lot odbył się zgodnie z planem, ale niczym szczególnym się nie wyróżniał.",
    "Stacja na orbicie spełniła minimalne oczekiwania, trudno coś więcej dodać.",
    "Czas spędzony w stanie nieważkości był interesujący, ale krótki.",
    "Obsługa była w miarę profesjonalna, choć bez entuzjazmu.",
    "Warunki codziennego pobytu w module były przeciętne."
]
positive_comments = [
    "Przelot przebiegł bez zarzutu, widoki zapierające dech w piersiach.",
    "Personel bardzo pomocny, poczułem się w pełni bezpiecznie.",
    "Eksperymenty naukowe w module okazały się fascynujące.",
    "Kabina komfortowa, z dużą przestrzenią i świetnym widokiem.",
    "Powrót na Ziemię był płynny, lądowanie perfekcyjne.",
    "Panel widokowy statku doskonale zaprojektowany dla fotografów.",
    "Program edukacyjny na pokładzie dostarczył wiele wiedzy."
]

# --- 4. Generowanie i wstawianie opinii ---
feedback_rows = []
now = datetime.now()

for trip_id, client_id, return_dt in completed_participants:
    # ok. 70% zostawia opinię
    if random.random() > 0.7:
        continue

    # jeśli lot miał incydent — oceny z 1–3 (z przewagą niskich)
    if trip_id in trips_with_incidents:
        rating = random.choices([1,2,3], weights=[0.5,0.3,0.2])[0]
    else:
        # inaczej z 3–5 (z przewagą wysokich)
        rating = random.choices([3,4,5], weights=[0.1,0.3,0.6])[0]

    # dobór komentarza do oceny
    if rating <= 2:
        comments = random.choice(negative_comments)
    elif rating == 3:
        comments = random.choice(neutral_comments)
    else:
        comments = random.choice(positive_comments)

    # data przesłania: między 1 a 30 dni po return_dt, ale nie później niż teraz
    raw_date = return_dt + timedelta(
        days=random.randint(1,30),
        hours=random.randint(0,23),
        minutes=random.randint(0,59),
        seconds=random.randint(0,59)
    )
    submitted_at = raw_date if raw_date < now else now - timedelta(
        days=random.randint(0,3),
        hours=random.randint(0,23),
        minutes=random.randint(0,59),
        seconds=random.randint(0,59)
    )

    feedback_rows.append((trip_id, client_id, rating, comments, submitted_at))

cursor.executemany(
    """
    INSERT INTO feedback (
        trip_id, client_id, rating, comments, submitted_at
    ) VALUES (%s, %s, %s, %s, %s)
    """,
    feedback_rows
)
con.commit()

## Generowanie incydentów

In [ ]:
# 1. Pobierz dane o lotach (trip_id, departure, return) oraz przypisanych pracownikach i klientach
cursor.execute("""
    SELECT trip_id, departure_datetime, return_datetime
    FROM trips
    WHERE status IN ('completed', 'in progress')
""")
trips_data = cursor.fetchall()  # lista: (trip_id, departure_datetime, return_datetime)

# Przypisani pracownicy:
cursor.execute("SELECT trip_id, employee_id FROM employee_assignments")
assignments = cursor.fetchall()  # lista: (trip_id, employee_id)
trip_to_employees = {}
for trip_id, emp_id in assignments:
    trip_to_employees.setdefault(trip_id, []).append(emp_id)

# Uczestnicy (klienci):
cursor.execute("SELECT trip_id, client_id FROM trip_participants")
parts = cursor.fetchall()  # lista: (trip_id, client_id)
trip_to_clients = {}
for trip_id, client_id in parts:
    trip_to_clients.setdefault(trip_id, []).append(client_id)

# 2. Przygotuj szablony incydentów, gdzie każdy wpis zawiera:
#    - description,
#    - category odpowiadające opisowi,
#    - requires_client (True/False),
#    - lista możliwych poziomów severity
incident_templates = [
    {
        "description": "Problemy z ciśnieniem w module medycznym, szybka reakcja lekarza.",
        "category": "medical",
        "requires_client": True,
        "possible_severities": ["medium", "high"]
    },
    {
        "description": "Awaria systemu orientacji – konieczne ręczne sterowanie przez pilota.",
        "category": "navigation",
        "requires_client": False,
        "possible_severities": ["medium", "high", "critical"]
    },
    {
        "description": "Utrata łączności z Ziemią na krótki okres, przywrócono po 15 minutach.",
        "category": "communication",
        "requires_client": False,
        "possible_severities": ["low", "medium"]
    },
    {
        "description": "Niewielki pożar w komorze silnikowej, ugaszono systemami automatycznymi.",
        "category": "equipment",
        "requires_client": False,
        "possible_severities": ["high", "critical"]
    },
    {
        "description": "Podejrzenie choroby lokomocyjnej u jednego z pasażerów, interwencja medyczna.",
        "category": "medical",
        "requires_client": True,
        "possible_severities": ["low", "medium"]
    },
    {
        "description": "Nieprawidłowy odczyt czujnika paliwa – wymagana weryfikacja inżynierska.",
        "category": "equipment",
        "requires_client": False,
        "possible_severities": ["medium", "high"]
    },
    {
        "description": "Zauważono podejrzany obiekt kosmiczny, wykonano dodatkową analizę.",
        "category": "navigation",
        "requires_client": False,
        "possible_severities": ["low", "medium"]
    },
    {
        "description": "Krótki alarm związany z poziomem tlenu, natychmiastowe sprawdzenie.",
        "category": "security",
        "requires_client": False,
        "possible_severities": ["medium", "high"]
    }
]

# 3. Wygeneruj incydenty
incident_rows = []
now = datetime.now()

for trip_id, departure_dt, return_dt in trips_data:
    # Losujemy 0-2 incydenty na lot
    n_incidents = random.randint(0, 2)
    for _ in range(n_incidents):
        # Wybierz szablon
        template = random.choice(incident_templates)
        description = template["description"]
        category = template["category"]
        requires_client = template["requires_client"]
        severity = random.choice(template["possible_severities"])

        # Wybierz moment wystąpienia incydentu:
        # – Jeśli lot zakończony (return_dt nie jest None), to losowo między departure_dt a return_dt
        # – Jeśli 'in progress', to losowo między departure_dt a teraz
        start_time = departure_dt
        end_time = return_dt if return_dt is not None else now
        if end_time <= start_time:
            incident_time = start_time + timedelta(hours=1)
        else:
            total_seconds = int((end_time - start_time).total_seconds())
            offset = random.randint(1, total_seconds - 1)
            incident_time = start_time + timedelta(seconds=offset)

        # Wybierz zgłaszającego pracownika (jeśli przypisanych brak, ustaw None)
        employees_for_trip = trip_to_employees.get(trip_id, [])
        reported_by = random.choice(employees_for_trip) if employees_for_trip else None

        # Jeśli w szablonie requires_client=True, losuj klienta z trip_to_clients
        involved_client = None
        if requires_client:
            clients_for_trip = trip_to_clients.get(trip_id, [])
            if clients_for_trip:
                involved_client = random.choice(clients_for_trip)

        incident_rows.append((
            trip_id,
            incident_time,
            reported_by,
            involved_client,
            category,
            description,
            severity
        ))

# 4. Wstawianie do tabeli incidents
cursor.executemany(
    """
    INSERT INTO incidents (
        trip_id, datetime_occurred, reported_by_employee,
        involved_client_id, category, description, severity
    ) VALUES (%s, %s, %s, %s, %s, %s, %s)
    """,
    incident_rows
)
con.commit()

## Generowanie kosztów dla każdej wyprawy

In [ ]:
cursor.execute("""
    SELECT trip_id, tt.base_price
    FROM trips t
    JOIN trip_types tt ON t.trip_type_id = tt.trip_type_id
""")
trip_base_prices = cursor.fetchall()  # lista: (trip_id, base_price)

# 2. Podstawowe kategorie kosztów
base_cost_templates = [
    {"description": "paliwo",                    "min_pct": 0.15, "max_pct": 0.25},
    {"description": "obsługa załogi",             "min_pct": 0.10, "max_pct": 0.18},
    {"description": "zapasy żywności",            "min_pct": 0.05, "max_pct": 0.10},
    {"description": "utrzymanie statku",          "min_pct": 0.08, "max_pct": 0.12},
    {"description": "ubezpieczenie",              "min_pct": 0.03, "max_pct": 0.06},
    {"description": "telekomunikacja",            "min_pct": 0.02, "max_pct": 0.04},
    {"description": "amortyzacja sprzętu",         "min_pct": 0.05, "max_pct": 0.10},
    {"description": "usługi medyczne",            "min_pct": 0.02, "max_pct": 0.05},
    {"description": "utrzymanie stacji startowej", "min_pct": 0.04, "max_pct": 0.08},
    {"description": "serwis i naprawy",           "min_pct": 0.06, "max_pct": 0.10}
]

# 3. Definiujemy dodatkowe koszty zależne od incydentów:
#    - jeśli wystąpił incydent "equipment", to dodajemy koszt naprawy sprzętu
#    - "medical" → dodatkowe usługi medyczne
#    - "security" → koszt operacji bezpieczeństwa
#    - "navigation" → koszty korekty kursu
#    - "communication" → koszty przywrócenia łączności
incident_cost_map = {
    "equipment":    {"description": "dodatkowa naprawa sprzętu (z incydentu)", "min_pct": 0.05, "max_pct": 0.10},
    "medical":      {"description": "dodatkowe usługi medyczne (z incydentu)", "min_pct": 0.03, "max_pct": 0.07},
    "security":     {"description": "operacje bezpieczeństwa (z incydentu)", "min_pct": 0.005, "max_pct": 0.01},
    "navigation":   {"description": "korekta kursu (z incydentu)", "min_pct": 0.005, "max_pct": 0.1},
    "communication":{"description": "przywrócenie łączności (z incydentu)", "min_pct": 0.02, "max_pct": 0.06},
}

# 4. Przygotowanie danych o incydentach: dla każdego trip_id zbieramy listę kategorii incydentu
cursor.execute("SELECT trip_id, category FROM incidents")
incidents_data = cursor.fetchall()  # lista: (trip_id, category)
trip_to_incidents = {}
for trip_id, category in incidents_data:
    trip_to_incidents.setdefault(trip_id, set()).add(category)

# 5. Generuj wiersze do costs
cost_rows = []

for trip_id, base_price in trip_base_prices:
    bp = float(base_price)  # rzutujemy na float

    # 5a. Wstawiamy wszystkie podstawowe kategorie kosztów
    for tpl in base_cost_templates:
        pct = random.uniform(tpl["min_pct"], tpl["max_pct"])
        cost_amount = round(bp * pct, 2)
        cost_rows.append((trip_id, tpl["description"], cost_amount))

    # 5b. Jeśli dla tego trip_id były jakieś incydenty, dodajemy odpowiednie pozycje kosztowe
    categories_for_trip = trip_to_incidents.get(trip_id, [])
    for cat in categories_for_trip:
        if cat in incident_cost_map:
            tpl_inc = incident_cost_map[cat]
            pct_inc = random.uniform(tpl_inc["min_pct"], tpl_inc["max_pct"])
            amount_inc = round(bp * pct_inc, 2)
            cost_rows.append((trip_id, tpl_inc["description"], amount_inc))

# 6. Wstawianie danych do tabeli costs
cursor.executemany(
    """
    INSERT INTO costs (
        trip_id, description, cost_amount
    ) VALUES (%s, %s, %s)
    """,
    cost_rows
)
con.commit()